In [12]:
import numpy as np
import pandas as pd
from sklearn.svm import SVR, SVC
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from skimage.io import imread
from skimage.transform import resize
import os

# Directories and file paths
train_dir = 'train_set\\images'
val_dir = 'val_set\\images'
test_dir = 'test_set\\images'
train_ann_file = 'train_annotation.csv'
val_ann_file = 'val_annotation.csv'
test_ann_file = 'val_annotation.csv'
img_width = 227
img_height = 227

# Function to load images and annotations
def load_data(dir_path, ann_file):
    images = []
    valences = []
    arousals = []
    emotions = []
    filenames = os.listdir(dir_path)
    anno = pd.read_csv(ann_file)
    
    for filename in filenames:
        img_path = os.path.join(dir_path, filename)
        img = imread(img_path)
        img_resized = resize(img, (img_height, img_width))
        
        
        row = anno[anno["filename"] == int(filename.split(".")[0])]
        if not row.empty:
            images.append(img_resized.flatten())  # Flatten image to 1D array
            valences.append(row["Valance"].values[0])
            arousals.append(row["Arousal"].values[0])
            emotions.append(row["Expression"].values[0])
    
    return np.array(images), np.array(valences), np.array(arousals), np.array(emotions)



In [1]:
from numba import cuda

device = cuda.get_current_device()
device.reset()


In [13]:
# Load training, validation, and test data
# X_train, y_train_valence, y_train_arousal, y_train_emotion = load_data(train_dir, train_ann_file)
X_val, y_val_valence, y_val_arousal, y_val_emotion = load_data(val_dir, val_ann_file)

In [14]:
X_test, y_test_valence, y_test_arousal, y_test_emotion = load_data(test_dir, val_ann_file)

In [15]:
# Standardize features
scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.fit_transform(X_val)
X_test_scaled = scaler.transform(X_test)

# One-hot encode emotions
one_hot_encoder = OneHotEncoder()
# y_train_emotion_encoded = one_hot_encoder.fit_transform(y_train_emotion.reshape(-1, 1))
y_val_emotion_encoded = one_hot_encoder.fit_transform(y_val_emotion.reshape(-1, 1))
y_test_emotion_encoded = one_hot_encoder.transform(y_test_emotion.reshape(-1, 1))

# Create and train SVR models for valence and arousal
svr_valence = SVR(kernel='rbf')
svr_arousal = SVR(kernel='rbf')
svc_emotion = SVC(kernel='rbf', probability=True)

In [16]:
svr_valence.fit(X_val_scaled, y_val_valence)
svr_arousal.fit(X_val_scaled, y_val_arousal)
svc_emotion.fit(X_val_scaled, y_val_emotion)

# Evaluate models on validation set
# valence_predictions = svr_valence.predict(X_val_scaled)
# arousal_predictions = svr_arousal.predict(X_val_scaled)
# emotion_predictions = svc_emotion.predict(X_val_scaled)

# # Calculate evaluation metrics
# valence_mse = np.mean((valence_predictions - y_val_valence) ** 2)
# arousal_mse = np.mean((arousal_predictions - y_val_arousal) ** 2)
# emotion_accuracy = np.mean(emotion_predictions == y_val_emotion)

# print(f'Valence MSE: {valence_mse:.4f}')
# print(f'Arousal MSE: {arousal_mse:.4f}')
# print(f'Emotion Accuracy: {emotion_accuracy:.4f}')

: 

In [ ]:
# Evaluate on test set
test_valence_predictions = svr_valence.predict(X_test_scaled)
test_arousal_predictions = svr_arousal.predict(X_test_scaled)
test_emotion_predictions = svc_emotion.predict(X_test_scaled)

# Calculate test set evaluation metrics
test_valence_mse = np.mean((test_valence_predictions - y_test_valence) ** 2)
test_arousal_mse = np.mean((test_arousal_predictions - y_test_arousal) ** 2)
test_emotion_accuracy = np.mean(test_emotion_predictions == y_test_emotion)

print(f'Test Valence MSE: {test_valence_mse:.4f}')
print(f'Test Arousal MSE: {test_arousal_mse:.4f}')
print(f'Test Emotion Accuracy: {test_emotion_accuracy:.4f}')